'''
OBJECTIVES:
1. Build WRS system
2. Build Structural BMP Solution evaluator
3. Identify minimum BMP solution front for:
   individual facilities
   facilities w/in departments
   facilities w/in city
   
PYTHON VERSION: 3.6.3  
SQLALCHEMY VERSION: 1.1.13

'''

In [1]:
import winsound

'''
Define basic SQLAlchemy items:
    declarative base object
    connection object
    session object
    DB tables
'''
#SQLAlchemy library items:
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String
from sqlalchemy import update, insert
from sqlalchemy import and_ #used in query.filter() to joing multiple where clauses
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship #http://docs.sqlalchemy.org/en/latest/orm/basic_relationships.html#relationship-patterns
from sqlalchemy import inspect

from SQLA_Base import Base #module containing declarative_base
from SQLA_conn_man import session, engine #module handling db and connection creation 

#Table definitions as SQLA classes:
from SQLA_DB_base_bmp_feasibility_test_results import Base_BMP_Feasibility_Test_Results as BBFTR
from SQLA_DB_base_bmp_feasibility_test_definitions import Base_BMP_Feasibility_Test_Definitions as BBFTD
from SQLA_DB_base_bmps import Base_BMPs
from SQLA_DB_combo_bmps import Combo_BMPs
from SQLA_DB_combo_bmp_feasibility_test_results import Combo_BMP_Feasibility_Test_Results as CBFTR
from SQLA_DB_expressions import Expressions
from SQLA_DB_facility_chars import Facility_Chars
from SQLA_DB_facility_monthly_rain import Facility_Monthly_Rain
from SQLA_DB_facility_risks import Facility_Risks
from SQLA_DB_facility_type_has_nel import Facility_Type_Has_NEL
from SQLA_DB_facility_types import Facility_Types
from SQLA_DB_feasibility_test_questions import Feasibility_Test_Questions as FTQ
from SQLA_DB_nel_sample_classes import NEL_Sample_Classes
from SQLA_DB_existing_pollutant_concentrations import Existing_Pollutant_Concentrations as ExPollConcs
from SQLA_DB_pollutant_removal_rates import Pollutant_Removal_Rates as PRR
from SQLA_DB_wrs_pollutant_risks import WRS_Pollutant_Risks
Base.metadata.create_all(engine, checkfirst=True) #create SQLA classes

'''
Dictionary of "SQLAlchemy where clause lambda functions" that importCSV uses to test record uniqueness.
used as the where clause in sqlalchemy queries, updates and deletes 
Form:
    {TableName:Lambda Function, TableName:Lambda Function, ...}
    
    TableName is the table name we want to define uniqueness test for
    Lambda Function is a SQLAlchemy query used to test record uniqueness. The function can take on any form 
        but must be made to evaluate the CSV row passed as a dictionary (CSVRowDict in this explanation):
        CSVRowDict: {FieldName:CSVColValue, DBTableFieldName:CSVColValue...} 
            Where: DBTableFieldName is the name of the field associated with the value at CSVColValue on the current row
                   CSVColValue: a value in the CSV's current row+column corresponding to the DBTableFieldName 
        *this assumes that field names are unique across table. if not, then method fails (maybe need to extend method?)
    FALSE: indicates that db table doesn't impose uniqueness on its records (other than its record id being unique)
        
e.g.: lambda myRowVal: Base.metadata.tables['people'].c['name'] == CSVRowDict['name']
        using lambda function in query will search for CSVRowDict's value for 'name' in the table people, field name 
if table has no record uniqueness requirement, then enter: TableName:False
'''
unqTests = {
    'facility_chars': lambda CSVRowDict: Base.metadata.tables['facility_chars'].c['Fac_Name'] == CSVRowDict['Fac_Name'],
    'facility_monthly_rain': False, #DB schema does not impose uniqueness on records in this table
    'facility_type_has_nel': False,
    'facility_risks': False,
    'facility_types': lambda CSVRowDict: Base.metadata.tables['facility_types'].c['Fac_Type'] == CSVRowDict['Fac_Type'],
    'nel_sample_classes': lambda CSVRowDict: Base.metadata.tables['nel_sample_classes'].c['nel_column']==CSVRowDict['nel_column'],
    'existing_pollutant_concentrations': False, #uniqueness not imposed for records in this table.
    'wrs_pollutant_risks': False #DB schema does not impose uniqueness on records in this table
}

import SQLA_main as SQLA_main #import main SQLAlchemy functions



Clearing old DB


In [2]:
'''
Define other custom modules

'''
import mod_Base_BMP_Eval as BBMP_Eval
import mod_Combo_BMP_Eval as CBMP_Eval
import mod_expression as Expr
import mod_importSpecial as importSpecial #special import functions are defined here
import mod_importCSV as importCSV #generic CSV importer ****IMPORTANT NOTE: function assumes csv in the utf-8-sig file format. weird things happen if its not in this format!!!


In [3]:
#import feasibillity questions, build feasibility expressions
importSpecial.importFeasibilityQuestionsCSV('Input_Files\\feasibility_test_questions.csv') 

#import base bmp information including:
  #1. imports definitions for cip costs, o&m costs, and BMP sizing to the expressions table
  #2. imports pollutant removal rates into pollutant_removal_rates table
  #3. creates a record in the base_bmps table using (1) and (2)
  #4. feasibility tests
importSpecial.importBaseBMPsCSV('Input_Files\\bmp_lego_piece.csv') 

#IMPORT BASIC FACILITY CHARS:
    #!!!!IMPORTANT!!!! This import must occur before other facility specific data is imported!
print ('\nImporting facility characteristics:')
importCSV.importCSV('Input_Files\\facility_chars.csv', unqTests)

#IMPORT PBP Appendix A1 data
print ('\nImporting PBP Appendix A1 data:')
importCSV.importCSV('Input_Files\\pbp_appxa1.csv', unqTests)

#IMPORT FACILITY RAINFALL EXTRACTED FROM http://rainfall.geography.hawaii.edu/downloads.html
print ('\nImporting Facility Rainfall Data:')
importCSV.importCSV('Input_Files\\FacilityRainfallData.csv', unqTests)

#IMPORT EFFLUENT LIMITS EXISTANCE FOR FACILITY TYPES: (either by Priority Based Plan, Table 3 or as City operational assignment)
#IF CSV HEADRS SETUP CORRECTLY, THEN THIS INSERTS NEL EXISTANCE DATA (0 OR 1) TO WRS_POLLUTANT TABLE 
#AND USES THE FACILITY_TYPE_HAS_NEL TO ASSOCIATE RECORD WITH FACILITY TYPE
print ('\nImporting Facility Type Has Effluent Limits:') #import into wrs_pollutant_risks table
importCSV.importCSV('Input_Files\\nel_exists_facility_types.csv', unqTests)

#IMPORT NEL CLASSIFICATION DATA (from PBP Appendix L)
print ('\nImporting NEL Classes')
importCSV.importCSV('Input_Files\\nel_pbp_appxl.csv', unqTests)

#IMPORT FACILITY RISKS:
print ('\nImporting Facility Risks')
#for future implementation:
    #The current process inserts fac risk and update existing_fac_char_id in Facility_chars table. this process thus creates
#dead records. a more sophisticated approach using sophisticated lambda function in unqTests would fix this
importCSV.importCSV('Input_Files\\facility_risks.csv', unqTests)

# #IMPORT FACILITY SAMPLING DATA
 #!!!IMPORTANT!!!! For now, we make none detects = 0 BUT this must be changed to detection limit, per DOH guidance.
print ('\nImporting Facilty Sampling data:')
importCSV.importCSV('Input_Files\\sample_data.csv', unqTests)


# for now, since we're developing, delete out all except 1st 2 facilities.
session.query(ExPollConcs).filter(ExPollConcs.facility_id >2).delete(synchronize_session = False) #http://docs.sqlalchemy.org/en/latest/orm/query.html#sqlalchemy.orm.query.Query.delete
session.query(Facility_Chars).filter(Facility_Chars.id >2).delete(synchronize_session = False) #http://docs.sqlalchemy.org/en/latest/orm/query.html#sqlalchemy.orm.query.Query.delete
session.commit #we chose not to sync session so need to commit before proceeding to requery or else you may get unpredictable resutls

session.commit()
winsound.Beep(250,1000)

Reading csv for import to Feasibility Questions

Reading csv record: Feas-1
Adding to variable dictionary: OFFSITE_SD_Exist

Reading csv record: Feas-2
Adding to variable dictionary: GW_Risk

Reading csv record: Feas-3
Adding to variable dictionary: GW_Risk

Reading csv record: Feas-4
Adding to variable dictionary: Soil_Type

Reading csv record: Feas-5
Adding to variable dictionary: Soil_Type

Reading csv record: Feas-6
Adding to variable dictionary: Soil_Type

Reading csv record: Feas-7
Adding to variable dictionary: Count_CB

Reading csv record: Feas-8
Adding to variable dictionary: Runoff_Type

Reading csv record: Feas-9
Adding to variable dictionary: TFMR_Exist

Reading csv record: Feas-10
Adding to variable dictionary: DS_SS_Exist

Reading csv record: Feas-11
Adding to variable dictionary: Fac_Slope

Reading csv record: Feas-12
Adding to variable dictionary: Can_Add_SD

Reading csv record: Feas-13
Adding to variable dictionary: Pave_Area
Adding to variable dictionary: BMP_Size(bas

imported records in  238  rows
associating records...

Importing Facilty Sampling data:
importing data in CSV rows...
imported records in  110  rows
associating records...


In [4]:
# EVALUATE BASE BMP FEASIBILITY at each facility
# Write results to the base_bmp_feasibility_test_results table.

print('\n******Evaluating Base BMP feasibility at facilities.******')
Expr.ResetEvalErrorCount() #RESET EXPRESION EVALUATOR ERROR COUNT
for aFac in session.query(Facility_Chars):
    print ('\n***Evaluating base bmp feasibiilty tests for facility: ', aFac.Fac_Name), ' ***'
    myBMPs = session.query(Base_BMPs)
    for aBMP in myBMPs:
        print ('\nEvaluating feasibility of base_bmp: ', aBMP.bmp_name, ' ID: ', aBMP.id)
        BBMP_Eval.Eval_base_bmp_feasibility_tests(aFac, aBMP)
session.commit
winsound.Beep(250,1000)
print ('*****************************************************************')
print ('* Completed evaluating Base BMP feasibility                     *')
if Expr.CountEvalErrors() >0:
    print (Expr.CountEvalErrors(), ' errors were encountered. Review output to identify location(s)')
    print ('Hint: expression evaluation error lines are prefixed by: FAULT!!!! Error occured while evaluating expression:')
else:
    print ('No errors detected.')
print ('*****************************************************************')


******Evaluating Base BMP feasibility at facilities.******

***Evaluating base bmp feasibiilty tests for facility:  Kalihi-Palama Bus & Paratransit Facility

Evaluating feasibility of base_bmp:  Hydrodynamic Separation  ID:  1

  Attempting eval of feasibility_test ID:  1
proccessing expression: Feas-1=OFFSITE_SD_Exist=='Yes'
    attempting to retrieve value for:  ('OFFSITE_SD_Exist', ['OFFSITE_SD_Exist', 'val', 'facility_chars', 'OFFSITE_SD_Exist', 'id', 'FLOAT'])
       QUERY RESULT: OFFSITE_SD_Exist='Yes'
  eval('Yes'=='Yes')=True
  Writing to DB Feasibility Test Result: True(1)
  Wrote to base_bmp_feasibility_test_results as recordID: 1

  Attempting eval of feasibility_test ID:  3
proccessing expression: Feas-3=GW_Risk!='High'
    attempting to retrieve value for:  ('GW_Risk', ['GW_Risk', 'val', 'facility_chars', 'GW_Risk', 'id', 'FLOAT'])
       QUERY RESULT: GW_Risk='High'
  eval('High'!='High')=False
  Writing to DB Feasibility Test Result: False(0)
  Wrote to base_bmp_feasibi

    attempting to retrieve value for:  ('Soil_Type', ['Soil_Type', 'val', 'facility_chars', 'Soil_Type', 'id', 'FLOAT'])
       QUERY RESULT: Soil_Type='Quarry'
  eval('Quarry'!='Rock')=True
  Writing to DB Feasibility Test Result: True(1)
  Wrote to base_bmp_feasibility_test_results as recordID: 28

  Attempting eval of feasibility_test ID:  13
proccessing expression: Feas-13=Pave_Area>BMP_Size(base_bmps~bmp_size_expression_id~bmp_name)
    attempting to retrieve value for:  ('Pave_Area', ['Pave_Area', 'val', 'facility_chars', 'Pave_Area', 'id', 'FLOAT'])
       QUERY RESULT: Pave_Area=867962.0
    attempting to retrieve value for:  ('BMP_Size(base_bmps~bmp_size_expression_id~bmp_name)', ['BMP_Size(base_bmps~bmp_size_expression_id~bmp_name)', 'dxp', 'base_bmps', 'bmp_size_expression_id', 'bmp_name', 'FLOAT'])
     This is a dynamic expression. Query for static expression using provided unique identifiers
       dynamic expression: BMP_Size(base_bmps~bmp_size_expression_id~bmp_name) = 

    attempting to retrieve value for:  ('OFFSITE_SD_Exist', ['OFFSITE_SD_Exist', 'val', 'facility_chars', 'OFFSITE_SD_Exist', 'id', 'FLOAT'])
       QUERY RESULT: OFFSITE_SD_Exist='No'
  eval('No'=='Yes')=False
  Writing to DB Feasibility Test Result: False(0)
  Wrote to base_bmp_feasibility_test_results as recordID: 50

  Attempting eval of feasibility_test ID:  3
proccessing expression: Feas-3=GW_Risk!='High'
    attempting to retrieve value for:  ('GW_Risk', ['GW_Risk', 'val', 'facility_chars', 'GW_Risk', 'id', 'FLOAT'])
       QUERY RESULT: GW_Risk='High'
  eval('High'!='High')=False
  Writing to DB Feasibility Test Result: False(0)
  Wrote to base_bmp_feasibility_test_results as recordID: 51

  Attempting eval of feasibility_test ID:  4
proccessing expression: Feas-4=Soil_Type!='Rock'
    attempting to retrieve value for:  ('Soil_Type', ['Soil_Type', 'val', 'facility_chars', 'Soil_Type', 'id', 'FLOAT'])
       QUERY RESULT: Soil_Type='Silty Clay Loam'
  eval('Silty Clay Loam'!='Ro

*****************************************************************
* Completed evaluating Base BMP feasibility                     *
No errors detected.
*****************************************************************


In [5]:
#Estimate Pollutant Effluent Limits
#Differentiate between wet and dry season limits (this only applies to NEL Column I vs II; I is dry season, II is wet season)
#FOR EACH FACILITY, calculate effluent limit by multiplying NEL_Column effluent limits by the facility type's NEL Exists [0,1] for each constituent 

import pandas as pd

def _HELPER_Get_pd_NEL_Values (recFac, WetORDry_Field):
    #helper function to get pandas df the NEL values based on the given facility's NEL_Column from the NEL_sample_Classes table
    #for the given facility, for the wet or dry season
    wrsid = session.query(NEL_Sample_Classes.wrs_pollutant_class_id).filter(WetORDry_Field == NEL_Sample_Classes.nel_column)
    q = session.query(
        WRS_Pollutant_Risks.wrs_tss,
        WRS_Pollutant_Risks.wrs_turbidity,
        WRS_Pollutant_Risks.wrs_p,
        WRS_Pollutant_Risks.wrs_n,
        WRS_Pollutant_Risks.wrs_nn,
        WRS_Pollutant_Risks.wrs_an,
        WRS_Pollutant_Risks.wrs_og,
        WRS_Pollutant_Risks.wrs_cu,
        WRS_Pollutant_Risks.wrs_zn,
        WRS_Pollutant_Risks.wrs_fe,
        WRS_Pollutant_Risks.wrs_phmin,
        WRS_Pollutant_Risks.wrs_phmax
    ).filter(WRS_Pollutant_Risks.id == wrsid.first()[0])
    return  pd.read_sql(q.statement,session.bind)     

def _HELPER_Get_pd_FacTypeHas_NEL (recFac):
    #helper function to get whether fac type requires NEL for each constituent (0 or 1) from the facility_type_has_nels table for the given recFac's fac_Type 
    wrsid = session.query(Facility_Type_Has_NEL.wrs_pollutant_limits_id).filter(fac.facility_type_id == Facility_Type_Has_NEL.facility_type_id)
    q = session.query(
        WRS_Pollutant_Risks.wrs_tss,
        WRS_Pollutant_Risks.wrs_turbidity,
        WRS_Pollutant_Risks.wrs_p,
        WRS_Pollutant_Risks.wrs_n,
        WRS_Pollutant_Risks.wrs_nn,
        WRS_Pollutant_Risks.wrs_an,
        WRS_Pollutant_Risks.wrs_og,
        WRS_Pollutant_Risks.wrs_cu,
        WRS_Pollutant_Risks.wrs_zn,
        WRS_Pollutant_Risks.wrs_fe,
        WRS_Pollutant_Risks.wrs_phmin,
        WRS_Pollutant_Risks.wrs_phmax    
    ).filter(WRS_Pollutant_Risks.id == wrsid.first()[0])
    return pd.read_sql(q.statement,session.bind) 

def GetNELs(recFac, OutputResults):
    #return 2 dataframes for the facility's NELs during wet and dry seasons
    #for each constituent, determine limit as intersection of Facility_Type_Has_NEL and NEL Column Values:
    #input: recFac: a facility_chars table record
    #input: OutputResults: bool indicating if we should output calculations and results
    #output: list of dictionaries: {}
    #get wet and dry season effluent limits for the facility's assigned NEL_Column:
    pd_NELColumn_Wet = _HELPER_Get_pd_NEL_Values(fac,fac.NEL_Column_Wet)
    pd_NELColumn_Dry = _HELPER_Get_pd_NEL_Values(fac,fac.NEL_Column_Dry)
    
    #get effluent limit requirements based on facility's facility_type (value is 0 or 1). 0 if not exist. 1 if exist:
    pd_NELRequired = _HELPER_Get_pd_FacTypeHas_NEL(fac)
    
    pd_FacNELs_Wet = pd_NELColumn_Wet.mul(pd_NELRequired,1)
    pd_FacNELs_Wet = pd_FacNELs_Wet.applymap(lambda x: float('nan') if x == 0 else x) #assign NaN values to any 0 element
    pd_FacNELs_Dry = pd_NELColumn_Dry.mul(pd_NELRequired,1)
    pd_FacNELs_Dry = pd_FacNELs_Dry.applymap(lambda x: float('nan') if x == 0 else x) #assign NaN values to any 0 element
    
    #for show-your-work purposes, make a summary dataframe:
    if OutputResults:
        df_new = pd.concat([pd.concat([pd.concat( [pd.concat([pd_NELColumn_Dry, pd_NELColumn_Wet]),pd_NELRequired]), pd_FacNELs_Dry]),pd_FacNELs_Wet])
        df_new.insert(0, 'description', 'x')
        df_new.iloc[[0],[0]]='Dry Season NELs* (Col. ' + fac.NEL_Column_Dry + '): '
        df_new.iloc[[1],[0]]='Wet Season NELs* (Col. ' + fac.NEL_Column_Wet + '): '
        df_new.iloc[[2],[0]]='NEL Exists**: '
        df_new.iloc[[3],[0]]='Facility Dry Season NELs: '
        df_new.iloc[[4],[0]]='Facility Wet Season NELs: '
        df_new = df_new.set_index('description')
        print ('\nSummary of Wet & Dry Season NEL Determination for: ', fac.Fac_Name)
        display(df_new)
        print ('   Notes: *Per PBP Appendix L; **Facility Type Requires this NEL (0: No; 1: Yes)')
        
    #multiply the NELs by the NEL requirements to get NEL values for this facility, for wet and dry seasons:
    pd_FacNELs_Wet.insert(0,'Facility_ID',fac.id)
    pd_FacNELs_Wet = pd_FacNELs_Wet.set_index('Facility_ID')
                          
    pd_FacNELs_Dry.insert(0,'Facility_ID',fac.id)  
    pd_FacNELs_Dry = pd_FacNELs_Dry.set_index('Facility_ID')
    return pd_FacNELs_Wet, pd_FacNELs_Dry

##### LATER, we can move the above code to a module and just keep this part in the notebook.        
pd_FacsNELs_Wet, pd_FacsNELs_Dry = pd.DataFrame(),  pd.DataFrame() #init wet and dry season nel dataframes 
for fac in session.query(Facility_Chars):
    wet,dry = GetNELs(fac,True)
    pd_FacsNELs_Wet = pd.concat([pd_FacsNELs_Wet, wet])
    pd_FacsNELs_Dry = pd.concat([pd_FacsNELs_Dry, dry])
display(pd_FacsNELs_Wet, pd_FacsNELs_Dry)



Summary of Wet & Dry Season NEL Determination for:  Kalihi-Palama Bus & Paratransit Facility


,wrs_tss,wrs_turbidity,wrs_p,wrs_n,wrs_nn,wrs_an,wrs_og,wrs_cu,wrs_zn,wrs_fe,wrs_phmin,wrs_phmax
description,,,,,,,,,,,,
Dry Season NELs* (Col. I):,30.0,5.5,0.06,0.38,0.09,NaN,15.0,6.0,22.0,1000.0,5.5,8.0
Wet Season NELs* (Col. II):,50.0,15.0,0.10,0.52,0.18,NaN,15.0,6.0,22.0,1000.0,5.5,8.0
NEL Exists**:,1.0,1.0,1.00,1.00,1.00,1.0,1.0,0.0,0.0,0.0,1.0,1.0
Facility Dry Season NELs:,30.0,5.5,0.06,0.38,0.09,NaN,15.0,NaN,NaN,NaN,5.5,8.0
Facility Wet Season NELs:,50.0,15.0,0.10,0.52,0.18,NaN,15.0,NaN,NaN,NaN,5.5,8.0


   Notes: *Per PBP Appendix L; **Facility Type Requires this NEL (0: No; 1: Yes)

Summary of Wet & Dry Season NEL Determination for:  Pearl City Bus Facility


,wrs_tss,wrs_turbidity,wrs_p,wrs_n,wrs_nn,wrs_an,wrs_og,wrs_cu,wrs_zn,wrs_fe,wrs_phmin,wrs_phmax
description,,,,,,,,,,,,
Dry Season NELs* (Col. I):,30.0,5.5,0.06,0.38,0.09,NaN,15.0,6.0,22.0,1000.0,5.5,8.0
Wet Season NELs* (Col. II):,50.0,15.0,0.10,0.52,0.18,NaN,15.0,6.0,22.0,1000.0,5.5,8.0
NEL Exists**:,1.0,1.0,1.00,1.00,1.00,1.0,1.0,0.0,0.0,0.0,1.0,1.0
Facility Dry Season NELs:,30.0,5.5,0.06,0.38,0.09,NaN,15.0,NaN,NaN,NaN,5.5,8.0
Facility Wet Season NELs:,50.0,15.0,0.10,0.52,0.18,NaN,15.0,NaN,NaN,NaN,5.5,8.0


   Notes: *Per PBP Appendix L; **Facility Type Requires this NEL (0: No; 1: Yes)


,wrs_tss,wrs_turbidity,wrs_p,wrs_n,wrs_nn,wrs_an,wrs_og,wrs_cu,wrs_zn,wrs_fe,wrs_phmin,wrs_phmax
Facility_ID,,,,,,,,,,,,
1,50.0,15.0,0.1,0.52,0.18,NaN,15.0,NaN,NaN,NaN,5.5,8.0
2,50.0,15.0,0.1,0.52,0.18,NaN,15.0,NaN,NaN,NaN,5.5,8.0


,wrs_tss,wrs_turbidity,wrs_p,wrs_n,wrs_nn,wrs_an,wrs_og,wrs_cu,wrs_zn,wrs_fe,wrs_phmin,wrs_phmax
Facility_ID,,,,,,,,,,,,
1,30.0,5.5,0.06,0.38,0.09,NaN,15.0,NaN,NaN,NaN,5.5,8.0
2,30.0,5.5,0.06,0.38,0.09,NaN,15.0,NaN,NaN,NaN,5.5,8.0


In [136]:
#let's work on calculating exceedance values
import math

def _HELPER_Get_pd_NEL_WetOrDry(SampleDate):
    '''
    Given the pd_FacsNELs_Wet and pd_FacsNELs_Dry dataframes, return one of the 2 depending on the given SampleDate
    Wet Season is from: January 1 through April 30 and November 1 through December 31
    Dry Season is from: May 1 through October 31

    Input: 
        SampleDate: date type object. Represents the date we want to get NEL data for.
    Global Variables assumed to exist:       
        pd_FacsNELs_Wet: pandas dataframe of wet season NELs
        pd_FacsNELs_Dry: pandas dataframe of dry season NELs    
    '''
    import datetime
    SampleDate = SampleDate.date()
    #Wet Season part 1 (between Jan. 1 of sampledate year and April 30 of sampledate year)
    if datetime.date(SampleDate.year, 1,1) <= SampleDate <= datetime.date(SampleDate.year, 4,30):
        return pd_FacsNELs_Wet
    #dry season (between May. 1 of sampledate year and October 31 of sampledate year)
    elif datetime.date(SampleDate.year, 5,1) <= SampleDate <= datetime.date(SampleDate.year, 10,31):
        return pd_FacsNELs_Dry
    #wet season part 2 (between November 1 and December 31 of sample date year i.e. rest of year not covered by conditions above)
    else:
        return pd_FacsNELs_Wet

def ExceedanceCalc(myRow, Constituent, FacID):
    '''
    pass in pandas row of: Sample_Date, PollunantSampleConcentrations
    pass in Pollutant constituent we need to calculate exceedance for
    pass in Facility ID we want to evaluate
    return: exceedance value

    function considers whether sample was taken during wet or dry season
    '''
    import numpy as np    
    if math.isnan(myRow['c_'+Constituent]): #if parameter value is NAN then return Nan
        return float('nan')
    elif math.isnan(_HELPER_Get_pd_NEL_WetOrDry(myRow['sample_date']).loc[FacID, 'wrs_'+Constituent]): #if NEL value is NAN then return Nan
        return float('nan')
    #if get to this point, then neither NEL or sample is NAN
    elif Constituent == 'phmin':
        try:
            ParamVal = max(0.0, _HELPER_Get_pd_NEL_WetOrDry(myRow['sample_date']).loc[FacID, 'wrs_'+Constituent] - myRow['c_'+Constituent])
        except:
            ParamVal = float('nan')
#             print(Constituent)
    elif Constituent == 'phmax':
        try:
            ParamVal = max(0.0,myRow['c_'+Constituent]  -  _HELPER_Get_pd_NEL_WetOrDry(myRow['sample_date']).loc[FacID, 'wrs_'+Constituent])
        except:
            ParamVal = float('nan')
#             print(Constituent)
    else:
        try:
            ParamVal = max(0.0,myRow['c_'+Constituent]  -  _HELPER_Get_pd_NEL_WetOrDry(myRow['sample_date']).loc[FacID, 'wrs_'+Constituent])
        except:
            ParamVal = float('nan')
#             print(Constituent)
    return ParamVal

###### stuff above this line should go into a module

FacID = 1
#get all sampling data for the given facility_id
q = session.query(ExPollConcs.id, ExPollConcs.facility_id.label('Facility_ID'), ExPollConcs.sample_date, 
        ExPollConcs.c_tss,
        ExPollConcs.c_turbidity,
        ExPollConcs.c_p,
        ExPollConcs.c_n,
        ExPollConcs.c_nn,
        ExPollConcs.c_an,
        ExPollConcs.c_og,
        ExPollConcs.c_cu,
        ExPollConcs.c_zn,
        ExPollConcs.c_fe,
        ExPollConcs.c_phmin,
        ExPollConcs.c_phmax  
         ).filter(ExPollConcs.facility_id == FacID)
pd_Concs = pd.read_sql(q.statement,session.bind) 

#tidy up the sampling data
from datetime import datetime
pd_Concs['sample_date'] = pd.to_datetime(pd_Concs['sample_date'], format="%m/%d/%Y")
pd_Concs = pd_Concs.applymap(lambda x: float('nan') if x is None else x) #assign NaN values to any None element

pd_Exceedances = pd_Concs.loc[:, ['Facility_ID','sample_date'] ] #copy sample date and facility id data to a new delta dataframe

pollLS = ['tss', 'turbidity', 'p', 'n', 'nn', 'an', 'og', 'cu', 'zn', 'fe', 'phmin', 'phmax'] #list of pollutant constituants we're trying to address

#for each pollutant constituent, calculate exceedance value. 
# if no exceedance, then report 0. report NaN sample result is NaN
for Constituent in pollLS:
    pd_Exceedances['c_' + Constituent]  = pd_Concs.apply(lambda x: ExceedanceCalc(x, Constituent, FacID), axis = 1)
display(pd_Exceedances)

,Facility_ID,sample_date,c_tss,c_turbidity,c_p,c_n,c_nn,c_an,c_og,c_cu,c_zn,c_fe,c_phmin,c_phmax
0,1,2017-04-19,72.0,NaN,NaN,1.910,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.00
1,1,2017-02-11,9.0,0.0,0.000,0.060,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.30
2,1,2016-12-04,30.0,56.2,0.200,0.390,0.000,NaN,0.0,NaN,NaN,NaN,0.0,0.20
3,1,2016-06-17,53.0,75.9,0.190,0.560,0.110,NaN,0.0,NaN,NaN,NaN,0.0,0.00
4,1,2015-02-20,0.0,2.5,0.076,1.310,0.010,NaN,0.0,NaN,NaN,NaN,0.0,0.54
5,1,2014-04-13,0.0,0.0,0.144,1.517,0.067,NaN,0.0,NaN,NaN,NaN,0.0,0.00
6,1,2013-03-09,113.0,9.4,0.055,0.679,0.059,NaN,0.0,NaN,NaN,NaN,0.0,0.09


In [151]:
'''
CALCULATE AGE FACTOR WEIGHTED AVERAGE FOR EACH CONSTITUENT:

Age factor acknowledges fact that more recent samples are a better representation of facility pollutant discharge 
(i.e. sampling data) and housekeeping-operations (i.e. inspections) realities. But, historic data as a whole also tells part 
of story (i.e. we want to dampen whipsaw effects that may occur if we only considered most recent data).

AF = exp(-SampleRank)
SampleRank = Newest sample = 1
              Second Newest sample = 2
              ...
              nth Newest Sample = n (out of n samples)
'''
import numpy as np
import math

# group dataframe by date
grouped = pd_Exceedances.groupby('sample_date')
#make age factored exceedence dataframe
pd_AFExceedances= pd.DataFrame(grouped.agg(np.max)) #write groups to new dataframe.
#establish sorted order. we want most recent data to be assigned lowest rank
pd_AFExceedances = pd_AFExceedances.sort_index(ascending=False) #sort by date, which is currently the data frame's index. most recent at top
pd_AFExceedances.reset_index(inplace=True) #index the dataframe we'll use the index as SampleRank

#define pd_headers to establish order that we want AF coludefine pd_headers to mn to be in within dataframe (do this before we actually add in AF to dataframe)
pd_headers = list(pd_AFExceedances) #get dataframe column list
pd_headers.insert(2,'AF') #add AF to the list at location we want AF column to appear

#make age factored exceedence data frame
pd_AFExceedances['AF'] = pd_AFExceedances.index #as initial step, write index to AF column. We'll use that value as the SAmpleRank 
pd_AFExceedances['AF'] = pd_AFExceedances['AF'].apply(lambda SRank: math.exp(-SRank)) #calculate age factor weight for the given SRank

#now rearrange AF column location using order given in pd_headers
pd_AFExceedances = pd_AFExceedances[pd_headers] #rearrange dataframe columns to have AF column where we want it

#calculate age factored exceedances (c_constituent * AF) for each constituent.
for Constituent in pollLS:
    pd_AFExceedances['c*AF_' + Constituent]  = pd_AFExceedances.apply(lambda row: (row['AF']*row['c_' + Constituent]), axis = 1)
print ('AF calculation summary:')
display(pd_AFExceedances)

#calculate age factor weighted average for each constituent. Put into new dataframe called pd_AFWExceedances
pd_AFWExceedances=pd.DataFrame({'Facility_ID':[FacID]})#insert facility id into pd_AFWExceedances
SumAF = pd_AFExceedances['AF'].sum() #get sum of AF weights
for Constituent in pollLS:
    pd_AFWExceedances['c*AF_' + Constituent]  = pd_AFExceedances['c*AF_' + Constituent].sum()/SumAF
print ('Age Factor Weighted Averages')
display(pd_AFWExceedances)


AF calculation summary:


,sample_date,Facility_ID,AF,c_tss,c_turbidity,c_p,c_n,c_nn,c_an,c_og,...,c*AF_p,c*AF_n,c*AF_nn,c*AF_an,c*AF_og,c*AF_cu,c*AF_zn,c*AF_fe,c*AF_phmin,c*AF_phmax
0,2017-04-19,1,1.000000,72.0,NaN,NaN,1.910,NaN,NaN,NaN,...,NaN,1.910000,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.000000
1,2017-02-11,1,0.367879,9.0,0.0,0.000,0.060,NaN,NaN,NaN,...,0.000000,0.022073,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.110364
2,2016-12-04,1,0.135335,30.0,56.2,0.200,0.390,0.000,NaN,0.0,...,0.027067,0.052781,0.000000,NaN,0.0,NaN,NaN,NaN,0.0,0.027067
3,2016-06-17,1,0.049787,53.0,75.9,0.190,0.560,0.110,NaN,0.0,...,0.009460,0.027881,0.005477,NaN,0.0,NaN,NaN,NaN,0.0,0.000000
4,2015-02-20,1,0.018316,0.0,2.5,0.076,1.310,0.010,NaN,0.0,...,0.001392,0.023993,0.000183,NaN,0.0,NaN,NaN,NaN,0.0,0.009890
5,2014-04-13,1,0.006738,0.0,0.0,0.144,1.517,0.067,NaN,0.0,...,0.000970,0.010221,0.000451,NaN,0.0,NaN,NaN,NaN,0.0,0.000000
6,2013-03-09,1,0.002479,113.0,9.4,0.055,0.679,0.059,NaN,0.0,...,0.000136,0.001683,0.000146,NaN,0.0,NaN,NaN,NaN,0.0,0.000223


Age Factor Weighted Averages


,Facility_ID,c*AF_tss,c*AF_turbidity,c*AF_p,c*AF_n,c*AF_nn,c*AF_an,c*AF_og,c*AF_cu,c*AF_zn,c*AF_fe,c*AF_phmin,c*AF_phmax
0,1,52.064543,7.246772,0.024691,1.296165,0.003959,0.0,0.0,0.0,0.0,0.0,0.0,0.093351


In [8]:
#CREATE COMBO BMPS USING BASE BMPS
#ALL POSSIBLE COMBOS WILL BE CREATED AND ADDED TO THE COMBO_BMPS TABLE
#MAXIMUM POLLUTANT REMOVAL RATES ARE DETERMINED BY IDENTIFYING 
#  THE BASE_BMP IN THE COMBO THAT PROVIDES THE HIGHEST REMOVAL RATE FOR A GIVEN POLLUTANT

import time
print ('get a coffee...this one takes a while!')
start_time = time.time()
CBMP_Eval.Make_ALL_bmp_base_option_combos()
session.commit()
print ('--- %s execution time in seconds ---' % (time.time() - start_time))

get a coffee...this one takes a while!
 Making BMP Combos of length: 1
 Find max pollutant removal rates for each BMP Combo of length:  1
  Made  13  combos
 Making BMP Combos of length: 2
 Find max pollutant removal rates for each BMP Combo of length:  2
  Made  78  combos
 Making BMP Combos of length: 3
 Find max pollutant removal rates for each BMP Combo of length:  3
  Made  286  combos
 Making BMP Combos of length: 4
 Find max pollutant removal rates for each BMP Combo of length:  4
  Made  715  combos
 Making BMP Combos of length: 5
 Find max pollutant removal rates for each BMP Combo of length:  5
  Made  1287  combos
 Making BMP Combos of length: 6
 Find max pollutant removal rates for each BMP Combo of length:  6
  Made  1716  combos
 Making BMP Combos of length: 7
 Find max pollutant removal rates for each BMP Combo of length:  7
  Made  1716  combos
 Making BMP Combos of length: 8
 Find max pollutant removal rates for each BMP Combo of length:  8
  Made  1287  combos
 Making

In [9]:
'''
Identify the feasible bmp combinations for each facility
Use base bmp feasibility results for each facility.
Put results into the combo_bmp_feasibility_test_results table
'''
import itertools     #https://docs.python.org/3/library/itertools.html    
import pandas as pd

from sqlalchemy import and_

def _Make_bmp_fingerprint(base_BMP_components):
    #create fingerprint of the passed list of base_bmp_ids
    #fingerprint is just a | separated list of ids of the base bmps that make up the combo bmp
    #corresponds to bmp_options table's bmp_fingerprint field
    #FORMAT: |bmp_option_base_component_id||bmp_option_base_component_id| w/ id's given in ascending order
    fingerprint = '|' + '|'.join(str(id) + '|' for id in base_BMP_components)
    return fingerprint

def Eval_FacBMPCombo(pd_basebmps, myFacility, bmpCombo):
    '''
    input:
        pdbasebmps: pandas built from a BBMP_Eval.evalFacility_BaseBMP dictionary list
                    assme that pandas is passed in w/ index is set as base_bmp_id
        myFacility: SQLA fac_chars record
        bmpCombo: list of base_bmp_ids that make up this combo
    
    #retrieve previously computed combo removal rate
    #calculate combo cip and om cost, insert/update database
    #calculate wrs reduction, insert/update database

    #return as pandas    
    '''    
    #get combo bmp pollutant removal rates into pandas 
    q = session.query(Combo_BMPs.bmp_fingerprint, Combo_BMPs.id.label('combos_bmp_id'), PRR.id.label('PRR_id'),
          PRR.r_tss, PRR.r_turbidity, PRR.r_p, PRR.r_n, PRR.r_nn, PRR.r_an,
          PRR.r_og, PRR.r_cu, PRR.r_zn, PRR.r_fe, PRR.r_phmin, PRR.r_phmax
        ).filter(Combo_BMPs.bmp_fingerprint == _Make_bmp_fingerprint(bmpCombo)).filter(
        Combo_BMPs.bmp_option_removal_rate_id == PRR.id)  
    pd_rr = pd.read_sql(q.statement,session.bind) 

    #use information in pd_rr to get CBFTR_record - make new record if necessary
    myCBFTR = Base.metadata.tables['combo_bmp_feasibility_test_results']
    myCBFTR_id = SQLA_main.insertupdateRec(myCBFTR,{'facility_id':myFacility.id, 'combo_bmps_id':pd_rr['combos_bmp_id'][0]},
                and_(
        myCBFTR.c['facility_id'] == myFacility.id,
        myCBFTR.c['combo_bmps_id'] == pd_rr['combos_bmp_id'][0]
                    ))
    session.flush()
    
    print (myCBFTR_id)
    
    #calculate WRS reduction
#     myFac_exWRSData = session.query(
    
    

    #get costs in pandas
    sumCIP = sum(pd_basebmps.loc[bmp_id,'calc_cip_cost'] for bmp_id in bmpCombo)
    sumOM = sum(pd_basebmps.loc[bmp_id,'calc_om_cost'] for bmp_id in bmpCombo)
    pd_sums = pd.DataFrame([{'calc_cip_cost':sumCIP, 'calc_om_cost': sumOM}])

    #merge combo bmp's removal rates and costs into 1 dataframe
    return pd.concat([pd_rr, pd_sums], axis = 1)
    
    

def Eval_FacBMPOptions(myFacility):
    #a wrapper around Eval_FacBMPCombo
    print('\n***Evaluating feasible bmp combos for facility: ', aFac.Fac_Name, '***')
    print ('****Evaluating feasibile base bmps****')
    df = pd.DataFrame(BBMP_Eval.evalFacility_BaseBMP(aFac, False)).set_index('base_bmp_id')
    display (df)   
    print ('****These are the feasible base bmps. I\'ll use them to make combos:****')
    df = df.loc[df['is_feasible'] == 1]
    display (df)
    feas_ls = df.index #send feasible base bmp ids to list
#     print (feas_ls)
# from SQLA_DB_combo_bmps import Combo_BMPs
# from SQLA_DB_combo_bmp_feasibility_test_results import Combo_BMP_Feasibility_Test_Results as CBFTR
    for CBOLen in range (1, len(feas_ls)+1): #+1 so it's inclusive of last count
        for combo in  itertools.combinations(feas_ls,CBOLen):
            print ('Here is a summary of the combo: ', list(combo))
            display(Eval_FacBMPCombo(df,myFacility, list(combo)))

            
def Eval_All_FacBMPOptions():
    print ('Evaluating feasibile BMP Options for each facility:')
    for aFac in session.query(Facility_Chars):
        Eval_FacBMPOptions(aFac)
    
Eval_All_FacBMPOptions()
session.commit()

Evaluating feasibile BMP Options for each facility:

***Evaluating feasible bmp combos for facility:  Pearl City Bus Facility ***
****Evaluating feasibile base bmps****


,base_bmp_name,calc_cip_cost,calc_om_cost,is_feasible
base_bmp_id,,,,
1,Hydrodynamic Separation,NaN,NaN,0
2,Enhanced Media Filtration (Replaceable Cartridge),NaN,NaN,0
3,Biofiltration (Vault),NaN,NaN,0
4,Media Filtration (Pressure),NaN,NaN,0
5,Bioinfiltration / Bioretention,NaN,NaN,0
6,Inlet Insert Unit,5000.0,5400.0,1
7,Sand Filtration,NaN,NaN,0
8,Coagulation Enhanced Treatment,NaN,NaN,0
9,Roofing,220000.0,0.0,1


****These are the feasible base bmps. I'll use them to make combos:****


,base_bmp_name,calc_cip_cost,calc_om_cost,is_feasible
base_bmp_id,,,,
6,Inlet Insert Unit,5000.0,5400.0,1
9,Roofing,220000.0,0.0,1
10,Paving and Curbing,3.0,0.0,1


Here is a summary of the combo:  [6]
1


...

,bmp_fingerprint,combos_bmp_id,PRR_id,r_tss,r_turbidity,r_p,r_n,r_nn,r_an,r_og,r_cu,r_zn,r_fe,r_phmin,r_phmax,calc_cip_cost,calc_om_cost
0,|6||10|,67,80,0.8,0.0,0.0,0.0,0.0,0.0,0.8,0.0,0.0,0.0,None,0.0,5003.0,5400.0


Here is a summary of the combo:  [9, 10]
13


,bmp_fingerprint,combos_bmp_id,PRR_id,r_tss,r_turbidity,r_p,r_n,r_nn,r_an,r_og,r_cu,r_zn,r_fe,r_phmin,r_phmax,calc_cip_cost,calc_om_cost
0,|9||10|,82,95,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,None,1.0,220003.0,0.0


Here is a summary of the combo:  [6, 9, 10]
14


,bmp_fingerprint,combos_bmp_id,PRR_id,r_tss,r_turbidity,r_p,r_n,r_nn,r_an,r_og,r_cu,r_zn,r_fe,r_phmin,r_phmax,calc_cip_cost,calc_om_cost
0,|6||9||10|,333,346,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,None,1.0,225003.0,5400.0


In [10]:
# session.close()
# engine.dispose()

In [11]:
# http://pythonhow.com/accessing-dataframe-columns-rows-and-cells/
import pandas as pd #import in pandas library
print ('#get csv data and read into pandas')
df1=pd.read_csv("http://pythonhow.com/wp-content/uploads/2016/01/Income_data.csv")
print (df1)
print ('#write new dataframe w/ index set to the "State" column in the csv')
df2=df1.set_index("State").copy()
print (df2)
print ('#extract a portion of the dataframe: States = Alaska to Arkansas; and Dates 2005:2007')
print (df2.loc["Alaska":"Arkansas","2005":"2007"])

print ('Get only certain States, using a list of states:')
getStates = ['Alaska', 'Arizona']
print (df2.loc[getStates])

print ('#slice a column:')
df2.loc[: , "2005"]
print ('get a cell:')
df2.loc['Alaska','2005']
print ('#get max of 2005 data')
print (df2.loc[:,'2005'].max())
print ('take 2005 column and put into list')
LS = df2['2005'].tolist() #this is a series. we use the .tolist() to convert from series to list
print (type(LS))


#get csv data and read into pandas


HTTPError: HTTP Error 403: Forbidden

In [ ]:
df = pd.DataFrame({'col1' : [1.0] * 5, 
                   'col2' : [2.0] * 5, 
                   'col3' : [3.0] * 5 }, index = range(1,6),)
display(df)
df2 = pd.DataFrame({'col1' : [10.0] * 5, 
                    'col2' : [100.0] * 5, 
                    'col3' : [1000.0] * 5 }, index = range(1,6),)
display(df2)
df.mul(df2, 0) # element by element multiplication no problems

In [ ]:
import datetime

# xmin = datetime.datetime.strptime('1/1/2018', "%m/%d/%Y").date()
# xmax = datetime.datetime.strptime('5/6/2018', "%m/%d/%Y").date()

# xmin <= datetime.date(2018,1,5) <= xmax

#     Wet Season is from: January 1 through April 30 and November 1 through December 31
#     Dry Season is from: May 1 through October 31

SampleDate = datetime.date(2018,11,1)

#Wet Season 1:
if datetime.date(SampleDate.year, 1,1) <= SampleDate <= datetime.date(SampleDate.year, 4,30):
    print ('ws 1')
elif datetime.date(SampleDate.year, 5,1) <= SampleDate <= datetime.date(SampleDate.year, 10,31):
    print ('dry')
else:
    print ('ws 2')
    
    
import numpy as np    
# np.max([float('nan'),0])
np.max([0,float('nan')])

if math.isnan(10)

In [ ]:
#import the pandas library
import pandas as pd

ipl_data = {'Team': ['Riders', 'Riders', 'Devils', 'Devils', 'Kings',
         'kings', 'Kings', 'Kings', 'Riders', 'Royals', 'Royals', 'Riders'],
         'Rank': [1, 2, 2, 3, 3,4 ,1 ,1,2 , 4,1,2],
         'Year': [2014,2015,2014,2015,2014,2015,2016,2017,2016,2014,2015,2017],
         'Points':[876,789,863,673,741,812,756,788,694,701,804,690]}
df = pd.DataFrame(ipl_data)

display (df)

display (df.groupby('Team').groups)


import numpy as np


grouped = df.groupby('Year')
print (grouped['Points'].agg(np.max))